In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
979881,2021-02-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
979882,2021-02-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
979883,2021-02-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
979884,2021-02-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
26711,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26713,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26715,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26717,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
26719,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
608224,2021-02-08,Arkansas,Arkansas,5001,1884,37.00,5000,Arkansas,AR,Arkansas,State,3017804
608226,2021-02-09,Arkansas,Arkansas,5001,1894,37.00,5000,Arkansas,AR,Arkansas,State,3017804
608228,2021-02-10,Arkansas,Arkansas,5001,1908,37.00,5000,Arkansas,AR,Arkansas,State,3017804
608230,2021-02-11,Arkansas,Arkansas,5001,1914,37.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
979881,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
979882,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
979883,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
979884,2021-02-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-12') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
388,2021-02-12,Snohomish,Washington,29705,516.00,53061,WA,County,822083,3613.38,62.77
774,2021-02-12,Cook,Illinois,464573,9669.00,17031,IL,County,5150233,9020.43,187.74
1159,2021-02-12,Orange,California,255254,3493.00,6059,CA,County,3175692,8037.74,109.99
1543,2021-02-12,Maricopa,Arizona,496426,8423.00,4013,AZ,County,4485414,11067.56,187.79
1927,2021-02-12,Los Angeles,California,1162294,18789.00,6037,CA,County,10039107,11577.66,187.16
...,...,...,...,...,...,...,...,...,...,...,...
979517,2021-02-12,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
979640,2021-02-12,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00
979732,2021-02-12,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
979820,2021-02-12,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
388,2021-02-12,Snohomish,Washington,29705,516.00,53061,WA,County,822083,3613.38,62.77,62.77,3613.38
774,2021-02-12,Cook,Illinois,464573,9669.00,17031,IL,County,5150233,9020.43,187.74,187.74,9020.43
1159,2021-02-12,Orange,California,255254,3493.00,6059,CA,County,3175692,8037.74,109.99,109.99,8037.74
1543,2021-02-12,Maricopa,Arizona,496426,8423.00,4013,AZ,County,4485414,11067.56,187.79,187.79,11067.56
1927,2021-02-12,Los Angeles,California,1162294,18789.00,6037,CA,County,10039107,11577.66,187.16,187.16,11577.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...
979517,2021-02-12,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
979640,2021-02-12,King,Texas,13,0.00,48269,TX,County,272,4779.41,0.00,0.00,4779.41
979732,2021-02-12,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
979820,2021-02-12,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
691755,2021-02-12,Clarke,Alabama,3376,46.00,1025,AL,County,23622,14291.76,194.73,194.73,14291.76,1
635835,2021-02-12,Hale,Alabama,2016,59.00,1065,AL,County,14651,13760.15,402.70,402.70,13760.15,2
498081,2021-02-12,Lowndes,Alabama,1283,44.00,1085,AL,County,9726,13191.45,452.40,452.40,13191.45,3
441862,2021-02-12,Etowah,Alabama,12914,297.00,1055,AL,County,102268,12627.61,290.41,290.41,12627.61,4
371046,2021-02-12,Franklin,Alabama,3941,75.00,1059,AL,County,31362,12566.16,239.14,239.14,12566.16,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606947,2021-02-12,Converse,Wyoming,939,15.00,56009,WY,County,13822,6793.52,108.52,108.52,6793.52,19
932631,2021-02-12,Platte,Wyoming,563,11.00,56031,WY,County,8393,6707.97,131.06,131.06,6707.97,20
837898,2021-02-12,Niobrara,Wyoming,147,2.00,56027,WY,County,2356,6239.39,84.89,84.89,6239.39,21
753572,2021-02-12,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
498081,2021-02-12,Lowndes,Alabama,1283,44.00,1085,AL,County,9726,13191.45,452.40,452.40,13191.45,3,1
635835,2021-02-12,Hale,Alabama,2016,59.00,1065,AL,County,14651,13760.15,402.70,402.70,13760.15,2,2
440562,2021-02-12,Clay,Alabama,1389,53.00,1027,AL,County,13235,10494.90,400.45,400.45,10494.90,23,3
536958,2021-02-12,Greene,Alabama,841,32.00,1063,AL,County,8111,10368.64,394.53,394.53,10368.64,25,4
497433,2021-02-12,Crenshaw,Alabama,1386,51.00,1041,AL,County,13772,10063.90,370.32,370.32,10063.90,31,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535665,2021-02-12,Hot Springs,Wyoming,358,3.00,56017,WY,County,4413,8112.40,67.98,67.98,8112.40,15,19
734928,2021-02-12,Uinta,Wyoming,1994,12.00,56041,WY,County,20226,9858.60,59.33,59.33,9858.60,6,20
753572,2021-02-12,Lincoln,Wyoming,1167,11.00,56023,WY,County,19830,5885.02,55.47,55.47,5885.02,22,21
211524,2021-02-12,Teton,Wyoming,3267,8.00,56039,WY,County,23464,13923.46,34.09,34.09,13923.46,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,62.77,3613.38,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
977564,2021-02-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
977565,2021-02-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
977566,2021-02-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
977567,2021-02-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
689121,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14291.76,45,1,2.00
689122,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,194.73,14291.76,45,1,0.00
689123,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,194.73,14291.76,45,1,1.00
689124,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,194.73,14291.76,45,1,4.00
689125,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,194.73,14291.76,45,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766056,2021-02-08,Crook,Wyoming,417,10.00,56011,WY,County,7584,5498.42,131.86,145.04,5498.42,6,23,0.00
766057,2021-02-09,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
766058,2021-02-10,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
766059,2021-02-11,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
496085,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13191.45,1,3,1.00,0.00
496086,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13191.45,1,3,0.00,0.00
496087,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13191.45,1,3,0.00,0.00
496088,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13191.45,1,3,0.00,0.00
496089,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,452.40,13191.45,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494460,2021-02-08,Albany,Wyoming,3831,11.00,56001,WY,County,38880,9853.40,28.29,28.29,9899.69,23,5,6.00,0.00
494461,2021-02-09,Albany,Wyoming,3832,11.00,56001,WY,County,38880,9855.97,28.29,28.29,9899.69,23,5,1.00,0.00
494462,2021-02-10,Albany,Wyoming,3838,11.00,56001,WY,County,38880,9871.40,28.29,28.29,9899.69,23,5,6.00,0.00
494463,2021-02-11,Albany,Wyoming,3845,11.00,56001,WY,County,38880,9889.40,28.29,28.29,9899.69,23,5,7.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-12') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
253529,2021-02-12,Imperial,California,26616,583.00,6025,CA,County,181215,14687.53,321.72,321.72,14687.53,1,2,27.00,0.00
443114,2021-02-12,Inyo,California,1144,34.00,6027,CA,County,18039,6341.81,188.48,188.48,6341.81,2,32,6.00,0.00
1927,2021-02-12,Los Angeles,California,1162294,18789.00,6037,CA,County,10039107,11577.66,187.16,187.16,11577.66,3,5,3159.00,131.00
56733,2021-02-12,Stanislaus,California,53842,894.00,6099,CA,County,550660,9777.72,162.35,162.35,9777.72,4,10,228.00,6.00
70290,2021-02-12,Tulare,California,45897,684.00,6107,CA,County,466195,9845.02,146.72,146.72,9845.02,5,9,115.00,12.00
26367,2021-02-12,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,600.00,47.00
401729,2021-02-12,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,0.00,0.00
536900,2021-02-12,Kings,California,21533,205.00,6031,CA,County,152940,14079.38,134.04,134.04,14079.38,8,3,90.00,0.00
45175,2021-02-12,San Joaquin,California,64969,992.00,6077,CA,County,762148,8524.46,130.16,130.16,8524.46,9,18,260.00,27.00
25681,2021-02-12,Fresno,California,92168,1291.00,6019,CA,County,999101,9225.09,129.22,129.22,9225.09,10,14,292.00,8.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
932182,2021-02-12,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00
253529,2021-02-12,Imperial,California,26616,583.00,6025,CA,County,181215,14687.53,321.72,321.72,14687.53,1,2,27.00,0.00
536900,2021-02-12,Kings,California,21533,205.00,6031,CA,County,152940,14079.38,134.04,134.04,14079.38,8,3,90.00,0.00
126545,2021-02-12,San Bernardino,California,280949,2308.00,6071,CA,County,2180085,12887.07,105.87,105.87,12887.07,13,4,881.00,65.00
1927,2021-02-12,Los Angeles,California,1162294,18789.00,6037,CA,County,10039107,11577.66,187.16,187.16,11577.66,3,5,3159.00,131.00
26367,2021-02-12,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,600.00,47.00
158985,2021-02-12,Kern,California,99570,677.00,6029,CA,County,900202,11060.85,75.21,75.21,11060.85,26,7,483.00,0.00
401729,2021-02-12,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,0.00,0.00
70290,2021-02-12,Tulare,California,45897,684.00,6107,CA,County,466195,9845.02,146.72,146.72,9845.02,5,9,115.00,12.00
56733,2021-02-12,Stanislaus,California,53842,894.00,6099,CA,County,550660,9777.72,162.35,162.35,9777.72,4,10,228.00,6.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
253529,2021-02-12,Imperial,California,26616,583.00,6025,CA,County,181215,14687.53,321.72,321.72,14687.53,1,2,27.00,0.00
443114,2021-02-12,Inyo,California,1144,34.00,6027,CA,County,18039,6341.81,188.48,188.48,6341.81,2,32,6.00,0.00
1927,2021-02-12,Los Angeles,California,1162294,18789.00,6037,CA,County,10039107,11577.66,187.16,187.16,11577.66,3,5,3159.00,131.00
56733,2021-02-12,Stanislaus,California,53842,894.00,6099,CA,County,550660,9777.72,162.35,162.35,9777.72,4,10,228.00,6.00
70290,2021-02-12,Tulare,California,45897,684.00,6107,CA,County,466195,9845.02,146.72,146.72,9845.02,5,9,115.00,12.00
26367,2021-02-12,Riverside,California,285414,3524.00,6065,CA,County,2470546,11552.67,142.64,142.64,11552.67,6,6,600.00,47.00
401729,2021-02-12,Merced,California,27701,375.00,6047,CA,County,277680,9975.87,135.05,135.05,9975.87,7,8,0.00,0.00
536900,2021-02-12,Kings,California,21533,205.00,6031,CA,County,152940,14079.38,134.04,134.04,14079.38,8,3,90.00,0.00
45175,2021-02-12,San Joaquin,California,64969,992.00,6077,CA,County,762148,8524.46,130.16,130.16,8524.46,9,18,260.00,27.00
25681,2021-02-12,Fresno,California,92168,1291.00,6019,CA,County,999101,9225.09,129.22,129.22,9225.09,10,14,292.00,8.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14687.53,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3312,02/08/21,Lassen,California,5489,18.00,6035,CA,County,30573,17953.75,58.88,62.15,17983.19,34,1,54.00,0.00
3313,02/09/21,Lassen,California,5492,18.00,6035,CA,County,30573,17963.56,58.88,62.15,17983.19,34,1,3.00,0.00
3314,02/10/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,6.00,1.00
3315,02/11/21,Lassen,California,5498,19.00,6035,CA,County,30573,17983.19,62.15,62.15,17983.19,34,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,4.00,0.00
1,03/21/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
2,03/22/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
3,03/23/20,Imperial,California,4,0.00,6025,CA,County,181215,2.21,0.00,321.72,14687.53,1,2,0.00,0.00
4,03/24/20,Imperial,California,9,0.00,6025,CA,County,181215,4.97,0.00,321.72,14687.53,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3386,02/08/21,Fresno,California,91184,1224.00,6019,CA,County,999101,9126.60,122.51,129.22,9225.09,10,14,240.00,0.00
3387,02/09/21,Fresno,California,91430,1254.00,6019,CA,County,999101,9151.23,125.51,129.22,9225.09,10,14,246.00,30.00
3388,02/10/21,Fresno,California,91730,1258.00,6019,CA,County,999101,9181.25,125.91,129.22,9225.09,10,14,300.00,4.00
3389,02/11/21,Fresno,California,91876,1283.00,6019,CA,County,999101,9195.87,128.42,129.22,9225.09,10,14,146.00,25.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)